# I. Linear codes for Inner Product Masking

### $n=2$ shares, $\ell=4$ bits

Parameters:

- $Z=(X + L_2Y_2, Y_2)=X\mathbf{G} + Y\mathbf{H}$ where $X, Y=(Y_2)$ and $Z$ are the sensitive variable, a mask and the protected variable, respectively. $\mathbf{G} = [1, 0]$ and $\mathbf{H} = [L_2, 1]$ are two generator matrices of codes $\mathcal{C}$ and $\mathcal{D}$, resp.
- $L_2\in \mathbb{F}_{2^\ell}\backslash\{0\}$, thus there are 15 linear codes for IPM
- Each element over $\mathbb{F}_{2^\ell}$ can be denoted as $\alpha^i$ where $i\in\{0, 1, \ldots, 14\}$, the corresponding irreducible polynomial is $g(\alpha) = \alpha^4 + \alpha +1$
- $\mathbf{H}^\perp = [1, L_2]$ is the generator matirx of dual code $\mathcal{D}^\perp$

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import pandas as pd # Pandas for tables
from IPython.display import Latex
from IPython.display import HTML

In [2]:
def read_f(file_name):
    """Reading weight enumerators."""
    with open(file_name, 'r') as fp:
        wd = fp.read().split("]\n")[:-1] # "\n"
        wd = np.array([list(map(int, re.findall(r"\d+", elem))) for elem in wd])
        
    return wd

## 1. Loading all weight enumerators

In [3]:
wd = read_f("weight_distrib_n2k4.txt") # Weight distribution

# print(wd.shape) # 16 entries: 15 for IPM codes and one for BKLC codes

### 1.1 Generating values

In [4]:
alpha_all = ['$\\alpha^{%d}$'%i for i in np.arange(len(wd))]
d_all = np.zeros(len(wd))
B_all = np.zeros(len(wd))
for i in range(len(wd)):
    d_all[i] = wd[i][2]
    B_all[i] = wd[i][3]

### 1.2 Defining styles of dataframe

See more setting of dataframe from https://mode.com/example-gallery/python_dataframe_styling/

In [5]:
# Set properties for th, td and caption elements in dataframe
th_props = [('font-size', '14px'), ('text-align', 'left'), ('font-weight', 'bold'), ('background-color', '#E0E0E0')]
td_props = [('font-size', '13px'), ('text-align', 'left'), ('min-width', '80px')]
cp_props = [('font-size', '16px'), ('text-align', 'center')]
# Set table styles
styles = [dict(selector="th", props=th_props), dict(selector="td", props=td_props), dict(selector="caption", props=cp_props)]
cm_1 = sns.light_palette("red", as_cmap=True)
cm_2 = sns.light_palette("purple", as_cmap=True, reverse=True)

In [6]:
df = pd.DataFrame({'$L_2$': alpha_all[:-1], '$d_{\mathcal{D}}^\perp$': d_all[:-1], '$B_{d_{\mathcal{D}}^\perp}$': B_all[:-1], 
                   'Weight Enumerators': wd[:-1]})

pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.width', 800)
(df.style
    .background_gradient(cmap=cm_1, subset=['$d_{\mathcal{D}}^\perp$','$B_{d_{\mathcal{D}}^\perp}$' ])
    .background_gradient(cmap=cm_2, subset=['$B_{d_{\mathcal{D}}^\perp}$' ])
    .set_caption('Tab. I All linear codes for IPM with $n=2$ shares over $\mathbb{F}_{2^4}$.')
    .set_table_styles(styles))


,$L_2$,$d_{\mathcal{D}}^\perp$,$B_{d_{\mathcal{D}}^\perp}$,Weight Enumerators
0,$\alpha^{0}$,2,4,"[0, 1, 2, 4, 4, 6, 6, 4, 8, 1]"
1,$\alpha^{1}$,2,3,"[0, 1, 2, 3, 3, 2, 4, 3, 5, 4, 6, 1, 7, 2]"
2,$\alpha^{2}$,2,2,"[0, 1, 2, 2, 3, 3, 4, 3, 5, 4, 6, 2, 7, 1]"
3,$\alpha^{3}$,2,1,"[0, 1, 2, 1, 3, 4, 4, 3, 5, 4, 6, 3]"
4,$\alpha^{4}$,3,4,"[0, 1, 3, 4, 4, 5, 5, 4, 6, 2]"
5,$\alpha^{5}$,3,3,"[0, 1, 3, 3, 4, 7, 5, 4, 7, 1]"
6,$\alpha^{6}$,3,4,"[0, 1, 3, 4, 4, 5, 5, 4, 6, 2]"
7,$\alpha^{7}$,3,4,"[0, 1, 3, 4, 4, 5, 5, 4, 6, 2]"
8,$\alpha^{8}$,3,4,"[0, 1, 3, 4, 4, 5, 5, 4, 6, 2]"
9,$\alpha^{9}$,3,4,"[0, 1, 3, 4, 4, 5, 5, 4, 6, 2]"


## 2. Optimal linear codes for IPM

### 2.1 Linear codes with $d_{\mathcal{D}}^\perp=3$

We focus on the the linear codes with greater $d_{\mathcal{D}}^\perp$, which are better in the sense of side-channel resistance (from our paper).

In [7]:
# Finding the indices of d_C=4 
d_index = []
d_C = 3
for i in range(len(wd)):
    if wd[i][2] == d_C:
        d_index.append(i)

#d_index = np.array(d_index)

In [8]:
print(len(d_index))

8


In [9]:
def highlight(s, threshold, column):
    is_min = pd.Series(data=False, index=s.index)
    is_min[column] = (s.loc[column] <= threshold)
    return ['background-color: gold' if is_min.any() else '' for v in is_min]

In [10]:
df_4 = pd.DataFrame({'$L_2$': np.array(alpha_all)[d_index], '$d_{\mathcal{D}}^\perp$': d_all[d_index], 
                     '$B_{d_{\mathcal{D}}^\perp}$': B_all[d_index], 'Weight Enumerators': wd[d_index]})
df_4 = df_4.sort_values(by=['$B_{d_{\mathcal{D}}^\perp}$'], ascending=True)

(df_4.style
    .apply(highlight, threshold=3, column=['$B_{d_{\mathcal{D}}^\perp}$'], axis=1)
    .background_gradient(cmap=cm_2, subset=['$B_{d_{\mathcal{D}}^\perp}$' ])
    .set_caption('Tab. II Linear codes for IPM with $d_{\mathcal{D}}^\perp=3$.')
    .set_table_styles(styles))

,$L_2$,$d_{\mathcal{D}}^\perp$,$B_{d_{\mathcal{D}}^\perp}$,Weight Enumerators
1,$\alpha^{5}$,3,3,"[0, 1, 3, 3, 4, 7, 5, 4, 7, 1]"
6,$\alpha^{10}$,3,3,"[0, 1, 3, 3, 4, 7, 5, 4, 7, 1]"
0,$\alpha^{4}$,3,4,"[0, 1, 3, 4, 4, 5, 5, 4, 6, 2]"
2,$\alpha^{6}$,3,4,"[0, 1, 3, 4, 4, 5, 5, 4, 6, 2]"
3,$\alpha^{7}$,3,4,"[0, 1, 3, 4, 4, 5, 5, 4, 6, 2]"
4,$\alpha^{8}$,3,4,"[0, 1, 3, 4, 4, 5, 5, 4, 6, 2]"
5,$\alpha^{9}$,3,4,"[0, 1, 3, 4, 4, 5, 5, 4, 6, 2]"
7,$\alpha^{11}$,3,4,"[0, 1, 3, 4, 4, 5, 5, 4, 6, 2]"


### 2.2 Optimal codes for IPM

As shown in our paper, the codes satifying two conditions are optimal:

- Maximizing $d_{\mathcal{D}}^\perp$, here $\max\{d_{\mathcal{D}}^\perp\} = 3$
- Minimizing $B_{d_{\mathcal{D}}^\perp}$, here $\min\{B_{d_{\mathcal{D}}^\perp}\} = 3$

Note that we use two leakage detection metrics **SNR** (signal-to-noise ratio) and **MI** (mutual information), and one leakage exploitation metric **SR** (success rate) to assess the side-channel resistance of IPM with different codes.

As a result of Tab. II, we conclude that the optimal codes for IPM are genetated by $\mathbf{H}=[L_2, 1]$ where $L_2\in\{\alpha^5, \alpha^{10}\}$.

The two optimal codes are equivalent and the generator matrix of the first one is: 
$$
\mathbf{H}_{optimal}=[\alpha^5, 1] \in \mathbb{F}_{2^4}^2
= \left(
 \begin{matrix}
   1 & 1 & 0 & 0 & 1 & 0 & 0 & 0  \\
   0 & 1 & 1 & 0 & 0 & 1 & 0 & 0  \\
   0 & 0 & 1 & 1 & 0 & 0 & 1 & 0  \\
   1 & 1 & 0 & 1 & 0 & 0 & 0 & 1  
  \end{matrix} 
\right) \in \mathbb{F}_2^{4\times 8}
$$ 

### 2.3 BKLC code with parameter $[8, 4, 4]$

BKLC is the short of Best Known Linear Code. 

**Note that the code $[8, 4, 4]$ is unique.**

In [11]:
bklc_index = [15]
cm_3 = sns.light_palette("red", as_cmap=True, reverse=True)
df_bklc = pd.DataFrame({'$L_2$': ['  --'], '$d_{\mathcal{D}}^\perp$': d_all[-1], '$B_{d_{\mathcal{D}}^\perp}$': B_all[-1], 
                        'Weight Enumerators': wd[bklc_index]})

(df_bklc.style
    .background_gradient(cmap=cm_3, subset=['$d_{\mathcal{D}}^\perp$', '$B_{d_{\mathcal{D}}^\perp}$'])
    .set_caption('Tab. III One BKLC code for IPM with $d_{\mathcal{D}}^\perp=4$.')
    .set_table_styles(styles))

,$L_2$,$d_{\mathcal{D}}^\perp$,$B_{d_{\mathcal{D}}^\perp}$,Weight Enumerators
0,--,4,14,"[0, 1, 4, 14, 8, 1]"


We can see that this BKLC code is better than all linear codes in IPM. It is interesting to notice that the BKLC code $[8, 4, 4]$ cannot be used in IPM, since it cannot be generated by $\mathbf{H}^\perp=[1, L_2]$ with any $L_2\in\mathbb{F}_{2^4}$.

The generator matrix of the dual code of BKLC code is: 
$$
\mathbf{H}_{BKLC}^\perp
= \left(
 \begin{matrix}
   1 & 1 & 1 & 0 & 1 & 0 & 0 & 0  \\
   1 & 1 & 0 & 1 & 0 & 1 & 0 & 0  \\
   1 & 0 & 1 & 1 & 0 & 0 & 1 & 0  \\
   0 & 1 & 1 & 1 & 0 & 0 & 0 & 1  
  \end{matrix} 
\right) \in \mathbb{F}_2^{4\times 8}
$$ 